In [ ]:
import pandas as pd
file_id = '1POXHpKuXsOHlrBaI2oKM44q46o-MVx7r'
url = f'https://drive.google.com/uc?id={file_id}&export=download'
output = 'data.csv'

gdown.download(url, output, quiet=False)

def load_data():
    return pd.read_csv(output)
df_encoded = load_data()

In [7]:
from sklearn.model_selection import train_test_split
X =  df_encoded.drop(columns=['emi_eligibility'])
y = df_encoded['emi_eligibility']
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)


from sklearn.preprocessing import LabelEncoder

# Create encoder
label_encoder = LabelEncoder()

# Fit and transform y values
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)


In [23]:
# LOgistic REgression
from sklearn.linear_model import LogisticRegression
log_model = LogisticRegression(max_iter=1000,class_weight='balanced')
log_model.fit(X_train,y_train)
y_pred_log = log_model.predict(X_test)
from sklearn.metrics import classification_report,roc_auc_score
print(classification_report(y_test,y_pred_log))
print("ROC-AUC:", roc_auc_score(y_test, log_model.predict_proba(X_test), multi_class='ovr'))


c:\Projects\emi_predict_ai\.venv\Lib\site-packages\sklearn\linear_model\_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 1000 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=1000).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


              precision    recall  f1-score   support

    Eligible       0.82      0.83      0.82     13866
   High_Risk       0.17      0.65      0.27      3247
Not_Eligible       0.99      0.84      0.91     61035

    accuracy                           0.83     78148
   macro avg       0.66      0.77      0.67     78148
weighted avg       0.93      0.83      0.87     78148

ROC-AUC: 0.9429106359886555


In [35]:
# Random forest
from sklearn.ensemble import RandomForestClassifier

# Create the model
rf_model = RandomForestClassifier(n_estimators=100, class_weight='balanced', random_state=42)

# Train the model
rf_model.fit(X_train, y_train)

# Predict on test data
y_pred_rf = rf_model.predict(X_test)


# Print metrics
print(classification_report(y_test, y_pred_rf))

# ROC-AUC (optional)
print("ROC-AUC:", roc_auc_score(y_test, rf_model.predict_proba(X_test), multi_class='ovr'))


              precision    recall  f1-score   support

    Eligible       0.93      0.93      0.93     13866
   High_Risk       0.92      0.04      0.07      3247
Not_Eligible       0.95      1.00      0.97     61035

    accuracy                           0.95     78148
   macro avg       0.93      0.66      0.66     78148
weighted avg       0.95      0.95      0.93     78148

ROC-AUC: 0.9814880641682241


In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
num_cols = ['monthly_salary', 'bank_balance', 'credit_score', 'total_expenses', 'debt_to_income_ratio', 'expense_to_income_ratio', 'emi_gap', 'credit_risk_score', 'employment_stability']
df_encoded[num_cols] = scaler.fit_transform(df_encoded[num_cols])

In [ ]:
# Salary × Credit Score
df_encoded['salary_credit_interaction'] = df_encoded['monthly_salary'] * df_encoded['credit_score']

# Bank Balance × EMI Gap
df_encoded['balance_emi_gap'] = df_encoded['bank_balance'] * df_encoded['emi_gap']

In [ ]:
df_encoded.to_csv('datasets/encoded_data.csv', index=False)

In [8]:
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, roc_auc_score

# Balance training data
smote = SMOTE(random_state=42)
X_train_bal, y_train_bal = smote.fit_resample(X_train, y_train_encoded) #type: ignore

# Train model
xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')
xgb_model.fit(X_train_bal, y_train_bal)

# Predict and evaluate
y_pred_xgb = xgb_model.predict(X_test)
print(classification_report(y_test_encoded, y_pred_xgb))
print("ROC-AUC:", roc_auc_score(y_test_encoded, xgb_model.predict_proba(X_test), multi_class='ovr'))

c:\Projects\emi_predict_ai\.venv\Lib\site-packages\xgboost\training.py:199: UserWarning: [23:20:16] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


              precision    recall  f1-score   support

           0       0.99      0.97      0.98     13866
           1       0.74      0.90      0.81      3247
           2       1.00      0.99      0.99     61035

    accuracy                           0.98     78148
   macro avg       0.91      0.95      0.93     78148
weighted avg       0.98      0.98      0.98     78148

ROC-AUC: 0.9979712170172425


In [6]:
from sklearn.tree import DecisionTreeClassifier

dt_model = DecisionTreeClassifier(random_state=42)
dt_model.fit(X_train, y_train_encoded)
y_pred_dt = dt_model.predict(X_test)
from sklearn.metrics import classification_report, roc_auc_score

print(classification_report(y_test_encoded, y_pred_dt))
print("ROC-AUC:", roc_auc_score(y_test_encoded, dt_model.predict_proba(X_test), multi_class='ovr'))


              precision    recall  f1-score   support

           0       0.97      0.96      0.97     13866
           1       0.70      0.72      0.71      3247
           2       0.99      0.99      0.99     61035

    accuracy                           0.98     78148
   macro avg       0.89      0.89      0.89     78148
weighted avg       0.98      0.98      0.98     78148

ROC-AUC: 0.9383242099254357


In [33]:
from sklearn.ensemble import HistGradientBoostingClassifier

hist_gb_model = HistGradientBoostingClassifier(max_iter=100, max_depth=3, random_state=42)
hist_gb_model.fit(X_train, y_train_encoded)
y_pred_hist = hist_gb_model.predict(X_test)
from sklearn.metrics import classification_report, roc_auc_score

print(classification_report(y_test_encoded, y_pred_hist))
print("ROC-AUC:", roc_auc_score(y_test_encoded, hist_gb_model.predict_proba(X_test), multi_class='ovr'))

              precision    recall  f1-score   support

           0       0.93      0.99      0.96     13866
           1       0.89      0.17      0.28      3247
           2       0.97      1.00      0.99     61035

    accuracy                           0.96     78148
   macro avg       0.93      0.72      0.74     78148
weighted avg       0.96      0.96      0.95     78148

ROC-AUC: 0.9929381640108464


In [17]:

# Drop classification target and regression target from features
X_reg = df_encoded.drop(columns=['emi_eligibility', 'max_monthly_emi'])

# Set regression target
y_reg = df_encoded['max_monthly_emi']
from sklearn.model_selection import train_test_split

X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(
    X_reg, y_reg, test_size=0.2, random_state=42
)
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_reg)
X_test_scaled = scaler.transform(X_test_reg)


In [18]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error
import numpy as np

def evaluate_regression_model(y_true, y_pred):
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))  # manual square root
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    mape = mean_absolute_percentage_error(y_true, y_pred)

    print(f"RMSE: {rmse:.2f}")
    print(f"MAE: {mae:.2f}")
    print(f"R² Score: {r2:.4f}")
    print(f"MAPE: {mape:.2%}")


In [19]:
from sklearn.linear_model import LinearRegression

lr_model = LinearRegression()
lr_model.fit(X_train_scaled, y_train_reg)
y_pred_lr = lr_model.predict(X_test_scaled)
evaluate_regression_model(y_test_reg, y_pred_lr)


RMSE: 0.00
MAE: 0.00
R² Score: 1.0000
MAPE: 0.00%


In [30]:
from xgboost import XGBRegressor
xgb_model_reg = XGBRegressor(n_estimators=100, random_state=42)
xgb_model_reg.fit(X_train_reg, y_train_reg)
y_pred_xgb = xgb_model_reg.predict(X_test_reg)
evaluate_regression_model(y_test_reg, y_pred_xgb)


RMSE: 264.01
MAE: 142.01
R² Score: 0.9986
MAPE: 7.18%


In [7]:
from sklearn.tree import DecisionTreeRegressor
dt_model = DecisionTreeRegressor(random_state=42)
dt_model.fit(X_train_reg, y_train_reg)
y_pred_dt = dt_model.predict(X_test_reg)
evaluate_regression_model(y_test_reg, y_pred_dt)


RMSE: 349.52
MAE: 109.57
R² Score: 0.9976
MAPE: 4.19%


In [5]:
from sklearn.ensemble import HistGradientBoostingRegressor

hist_gb_reg = HistGradientBoostingRegressor(max_iter=100, max_depth=3, random_state=42)
hist_gb_reg.fit(X_train_reg, y_train_reg)
y_pred_hist = hist_gb_reg.predict(X_test_reg)
evaluate_regression_model(y_test_reg, y_pred_hist)


RMSE: 564.85
MAE: 357.67
R² Score: 0.9938
MAPE: 25.99%


In [ ]:
import pandas as pd
import numpy as np
import mlflow
import mlflow.sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import (
    classification_report, roc_auc_score,
    mean_squared_error, mean_absolute_error,
    r2_score, mean_absolute_percentage_error
)

# Load data
df_encoded = pd.read_csv('datasets/encoded_data.csv')

# Classification setup
X = df_encoded.drop(columns=['emi_eligibility'])
y = df_encoded['emi_eligibility']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# Regression setup
X_reg = df_encoded.drop(columns=['emi_eligibility', 'max_monthly_emi'])
y_reg = df_encoded['max_monthly_emi']
X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(X_reg, y_reg, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_reg)
X_test_scaled = scaler.transform(X_test_reg)

In [13]:
print(list(label_encoder.classes_))

['Eligible', 'High_Risk', 'Not_Eligible']


In [39]:
import mlflow
mlflow.set_experiment("EMI Classification Models")

2025/10/29 23:49:26 INFO mlflow.tracking.fluent: Experiment with name 'EMI Classification Models' does not exist. Creating a new experiment.


<Experiment: artifact_location='file:///c:/Projects/emi_predict_ai/mlruns/958227794677363910', creation_time=1761761966634, experiment_id='958227794677363910', last_update_time=1761761966634, lifecycle_stage='active', name='EMI Classification Models', tags={}>

In [42]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

with mlflow.start_run(run_name="Logistic Regression"):
    log_model = LogisticRegression(max_iter=1000, class_weight='balanced')
    log_model.fit(X_train, y_train)
    y_pred = log_model.predict(X_test)
    y_pred_proba = log_model.predict_proba(X_test)

    mlflow.log_params(log_model.get_params())
    mlflow.log_metric("Accuracy", accuracy_score(y_test, y_pred))
    mlflow.log_metric("Precision", precision_score(y_test, y_pred, average='weighted'))
    mlflow.log_metric("Recall", recall_score(y_test, y_pred, average='weighted'))
    mlflow.log_metric("F1_Score", f1_score(y_test, y_pred, average='weighted'))
    mlflow.log_metric("ROC_AUC", roc_auc_score(y_test, y_pred_proba, multi_class='ovr'))

    mlflow.sklearn.log_model(log_model, "model", input_example=pd.DataFrame(X_test[:1]))

c:\Projects\emi_predict_ai\.venv\Lib\site-packages\sklearn\linear_model\_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 1000 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=1000).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
2025/10/30 00:05:03 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
c:\Projects\emi_predict_ai\.venv\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and 

In [43]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

with mlflow.start_run(run_name="Random Forest Classifier"):
    rf_model = RandomForestClassifier(n_estimators=100, class_weight='balanced', random_state=42)
    rf_model.fit(X_train, y_train)
    y_pred = rf_model.predict(X_test)
    y_pred_proba = rf_model.predict_proba(X_test)

    mlflow.log_params(rf_model.get_params())
    mlflow.log_metric("Accuracy", accuracy_score(y_test, y_pred))
    mlflow.log_metric("Precision", precision_score(y_test, y_pred, average='weighted'))
    mlflow.log_metric("Recall", recall_score(y_test, y_pred, average='weighted'))
    mlflow.log_metric("F1_Score", f1_score(y_test, y_pred, average='weighted'))
    mlflow.log_metric("ROC_AUC", roc_auc_score(y_test, y_pred_proba, multi_class='ovr'))

    mlflow.sklearn.log_model(rf_model, "model", input_example=pd.DataFrame(X_test[:1]))

2025/10/30 00:51:49 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
c:\Projects\emi_predict_ai\.venv\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(


In [44]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

mlflow.set_experiment("EMI Classification Models")
with mlflow.start_run(run_name="XGBoost Classifier"):
    xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')
    xgb_model.fit(X_train_bal, y_train_bal)
    y_pred = xgb_model.predict(X_test)
    y_pred_proba = xgb_model.predict_proba(X_test)

    mlflow.log_params(xgb_model.get_params())
    mlflow.log_metric("Accuracy", accuracy_score(y_test_encoded, y_pred))
    mlflow.log_metric("Precision", precision_score(y_test_encoded, y_pred, average='weighted'))
    mlflow.log_metric("Recall", recall_score(y_test_encoded, y_pred, average='weighted'))
    mlflow.log_metric("F1_Score", f1_score(y_test_encoded, y_pred, average='weighted'))
    mlflow.log_metric("ROC_AUC", roc_auc_score(y_test_encoded, y_pred_proba, multi_class='ovr'))

    mlflow.sklearn.log_model(xgb_model, "model", input_example=pd.DataFrame(X_test[:1]))

c:\Projects\emi_predict_ai\.venv\Lib\site-packages\xgboost\training.py:199: UserWarning: [00:54:45] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
2025/10/30 00:54:54 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
c:\Projects\emi_predict_ai\.venv\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Miss

In [45]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

mlflow.set_experiment("EMI Classification Models")
with mlflow.start_run(run_name="Decision Tree Classifier"):
    dt_model = DecisionTreeClassifier(random_state=42)
    dt_model.fit(X_train, y_train_encoded)
    y_pred = dt_model.predict(X_test)
    y_pred_proba = dt_model.predict_proba(X_test)

    mlflow.log_params(dt_model.get_params())
    mlflow.log_metric("Accuracy", accuracy_score(y_test_encoded, y_pred))
    mlflow.log_metric("Precision", precision_score(y_test_encoded, y_pred, average='weighted'))
    mlflow.log_metric("Recall", recall_score(y_test_encoded, y_pred, average='weighted'))
    mlflow.log_metric("F1_Score", f1_score(y_test_encoded, y_pred, average='weighted'))
    mlflow.log_metric("ROC_AUC", roc_auc_score(y_test_encoded, y_pred_proba, multi_class='ovr'))

    mlflow.sklearn.log_model(dt_model, "model", input_example=pd.DataFrame(X_test[:1]))

2025/10/30 00:55:48 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
c:\Projects\emi_predict_ai\.venv\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
c:\Projects\emi_predict_ai\.venv\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer 

In [46]:
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

mlflow.set_experiment("EMI Classification Models")
with mlflow.start_run(run_name="HistGradientBoosting Classifier"):
    hist_gb_model = HistGradientBoostingClassifier(max_iter=100, max_depth=3, random_state=42)
    hist_gb_model.fit(X_train, y_train_encoded)
    y_pred = hist_gb_model.predict(X_test)
    y_pred_proba = hist_gb_model.predict_proba(X_test)

    mlflow.log_params(hist_gb_model.get_params())
    mlflow.log_metric("Accuracy", accuracy_score(y_test_encoded, y_pred))
    mlflow.log_metric("Precision", precision_score(y_test_encoded, y_pred, average='weighted'))
    mlflow.log_metric("Recall", recall_score(y_test_encoded, y_pred, average='weighted'))
    mlflow.log_metric("F1_Score", f1_score(y_test_encoded, y_pred, average='weighted'))
    mlflow.log_metric("ROC_AUC", roc_auc_score(y_test_encoded, y_pred_proba, multi_class='ovr'))

    mlflow.sklearn.log_model(hist_gb_model, "model", input_example=pd.DataFrame(X_test[:1]))

2025/10/30 00:56:31 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
c:\Projects\emi_predict_ai\.venv\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
c:\Projects\emi_predict_ai\.venv\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer 

In [47]:
mlflow.set_experiment("EMI Regression Models")


2025/10/30 00:57:27 INFO mlflow.tracking.fluent: Experiment with name 'EMI Regression Models' does not exist. Creating a new experiment.


<Experiment: artifact_location='file:///c:/Projects/emi_predict_ai/mlruns/884078883015768601', creation_time=1761766047917, experiment_id='884078883015768601', last_update_time=1761766047917, lifecycle_stage='active', name='EMI Regression Models', tags={}>

In [48]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error
import numpy as np

with mlflow.start_run(run_name="Linear Regression"):
    lr_model = LinearRegression()
    lr_model.fit(X_train_scaled, y_train_reg)
    y_pred = lr_model.predict(X_test_scaled)

    mlflow.log_params(lr_model.get_params())
    mlflow.log_metric("RMSE", np.sqrt(mean_squared_error(y_test_reg, y_pred)))
    mlflow.log_metric("MAE", mean_absolute_error(y_test_reg, y_pred))
    mlflow.log_metric("R2", r2_score(y_test_reg, y_pred))
    mlflow.log_metric("MAPE", mean_absolute_percentage_error(y_test_reg, y_pred))

    mlflow.sklearn.log_model(lr_model, "model", input_example=pd.DataFrame(X_test_scaled[:1]))

2025/10/30 00:59:57 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


In [49]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error
import numpy as np

with mlflow.start_run(run_name="XGBoost Regressor"):
    xgb_model_reg = XGBRegressor(n_estimators=100, random_state=42)
    xgb_model_reg.fit(X_train_reg, y_train_reg)
    y_pred = xgb_model_reg.predict(X_test_reg)

    mlflow.log_params(xgb_model_reg.get_params())
    mlflow.log_metric("RMSE", np.sqrt(mean_squared_error(y_test_reg, y_pred)))
    mlflow.log_metric("MAE", mean_absolute_error(y_test_reg, y_pred))
    mlflow.log_metric("R2", r2_score(y_test_reg, y_pred))
    mlflow.log_metric("MAPE", mean_absolute_percentage_error(y_test_reg, y_pred))

    mlflow.sklearn.log_model(xgb_model_reg, "model", input_example=pd.DataFrame(X_test_reg[:1]))

2025/10/30 01:01:42 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
c:\Projects\emi_predict_ai\.venv\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(


In [50]:
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error
import numpy as np

with mlflow.start_run(run_name="SVR"):
    svr_model = SVR()
    svr_model.fit(X_train_scaled, y_train_reg)
    y_pred = svr_model.predict(X_test_scaled)

    mlflow.log_params(svr_model.get_params())
    mlflow.log_metric("RMSE", np.sqrt(mean_squared_error(y_test_reg, y_pred)))
    mlflow.log_metric("MAE", mean_absolute_error(y_test_reg, y_pred))
    mlflow.log_metric("R2", r2_score(y_test_reg, y_pred))
    mlflow.log_metric("MAPE", mean_absolute_percentage_error(y_test_reg, y_pred))

    mlflow.sklearn.log_model(svr_model, "model", input_example=pd.DataFrame(X_test_scaled[:1]))

: 

In [5]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error
import numpy as np

with mlflow.start_run(run_name="Decision Tree Regressor"):
    dt_model = DecisionTreeRegressor(random_state=42)
    dt_model.fit(X_train_reg, y_train_reg)
    y_pred = dt_model.predict(X_test_reg)

    mlflow.log_params(dt_model.get_params())
    mlflow.log_metric("RMSE", np.sqrt(mean_squared_error(y_test_reg, y_pred)))
    mlflow.log_metric("MAE", mean_absolute_error(y_test_reg, y_pred))
    mlflow.log_metric("R2", r2_score(y_test_reg, y_pred))
    mlflow.log_metric("MAPE", mean_absolute_percentage_error(y_test_reg, y_pred))

    mlflow.sklearn.log_model(dt_model, "model", input_example=pd.DataFrame(X_test_reg[:1]))

2025/10/30 01:13:39 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
c:\Projects\emi_predict_ai\.venv\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(


🏃 View run Decision Tree Regressor at: http://127.0.0.1:5000/#/experiments/884078883015768601/runs/4bfa1b4ee9bf4d918198bace20b9fa3a
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/884078883015768601


In [3]:
mlflow.set_tracking_uri("http://127.0.0.1:5000/")

In [4]:
mlflow.set_tracking_uri("http://127.0.0.1:5000/")
mlflow.set_experiment("EMI Regression Models")

<Experiment: artifact_location='file:///c:/Projects/emi_predict_ai/mlruns/884078883015768601', creation_time=1761766047917, experiment_id='884078883015768601', last_update_time=1761766047917, lifecycle_stage='active', name='EMI Regression Models', tags={}>

In [6]:
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error
import numpy as np

mlflow.set_experiment("EMI Regression Models")
with mlflow.start_run(run_name="HistGradientBoosting Regressor"):
    hist_gb_reg = HistGradientBoostingRegressor(max_iter=100, max_depth=3, random_state=42)
    hist_gb_reg.fit(X_train_reg, y_train_reg)
    y_pred = hist_gb_reg.predict(X_test_reg)

    mlflow.log_params(hist_gb_reg.get_params())
    mlflow.log_metric("RMSE", np.sqrt(mean_squared_error(y_test_reg, y_pred)))
    mlflow.log_metric("MAE", mean_absolute_error(y_test_reg, y_pred))
    mlflow.log_metric("R2", r2_score(y_test_reg, y_pred))
    mlflow.log_metric("MAPE", mean_absolute_percentage_error(y_test_reg, y_pred))

    mlflow.sklearn.log_model(hist_gb_reg, "model", input_example=pd.DataFrame(X_test_reg[:1]))

2025/10/30 01:14:27 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
c:\Projects\emi_predict_ai\.venv\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(


🏃 View run HistGradientBoosting Regressor at: http://127.0.0.1:5000/#/experiments/884078883015768601/runs/8367f82dee4c405cba7d7a15e8aaafe8
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/884078883015768601


In [ ]:
mlflow.register_model(
    model_uri="runs:/cfe97507737c468d96328c44d88ba10e/model",
    name="EMI_Classifier_XGBoost_Updated"
)

Registered model 'EMI_Classifier_XGBoost' already exists. Creating a new version of this model...
2025/10/31 00:17:15 WARNING mlflow.tracking._model_registry.fluent: Run with id cfe97507737c468d96328c44d88ba10e has no artifacts at artifact path 'model', registering model based on models:/m-489da889889443a8af1433bbbad454b8 instead
Created version '3' of model 'EMI_Classifier_XGBoost'.


<ModelVersion: aliases=[], creation_timestamp=1761850035237, current_stage='None', deployment_job_state=None, description=None, last_updated_timestamp=1761850035237, metrics=[<Metric: dataset_digest=None, dataset_name=None, key='Accuracy', model_id='m-489da889889443a8af1433bbbad454b8', run_id='cfe97507737c468d96328c44d88ba10e', step=0, timestamp=1761765894106, value=0.9825203460101346>,
 <Metric: dataset_digest=None, dataset_name=None, key='F1_Score', model_id='m-489da889889443a8af1433bbbad454b8', run_id='cfe97507737c468d96328c44d88ba10e', step=0, timestamp=1761765894139, value=0.9833273662190362>,
 <Metric: dataset_digest=None, dataset_name=None, key='Precision', model_id='m-489da889889443a8af1433bbbad454b8', run_id='cfe97507737c468d96328c44d88ba10e', step=0, timestamp=1761765894119, value=0.984853183694632>,
 <Metric: dataset_digest=None, dataset_name=None, key='Recall', model_id='m-489da889889443a8af1433bbbad454b8', run_id='cfe97507737c468d96328c44d88ba10e', step=0, timestamp=176176

In [8]:
mlflow.register_model(
    model_uri="runs:/c6efeb5a1e844a378d0b6051d1f52497/model",
    name="EMI_Regressor_XGBoost"
)

Successfully registered model 'EMI_Regressor_XGBoost'.
2025/10/30 01:28:29 WARNING mlflow.tracking._model_registry.fluent: Run with id c6efeb5a1e844a378d0b6051d1f52497 has no artifacts at artifact path 'model', registering model based on models:/m-064cf115b7264f9ba051e9161f74b1e3 instead
2025/10/30 01:28:29 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: EMI_Regressor_XGBoost, version 1
Created version '1' of model 'EMI_Regressor_XGBoost'.


<ModelVersion: aliases=[], creation_timestamp=1761767909435, current_stage='None', deployment_job_state=<ModelVersionDeploymentJobState: current_task_name='', job_id='', job_state='DEPLOYMENT_JOB_CONNECTION_STATE_UNSPECIFIED', run_id='', run_state='DEPLOYMENT_JOB_RUN_STATE_UNSPECIFIED'>, description='', last_updated_timestamp=1761767909435, metrics=None, model_id=None, name='EMI_Regressor_XGBoost', params=None, run_id='c6efeb5a1e844a378d0b6051d1f52497', run_link='', source='models:/m-064cf115b7264f9ba051e9161f74b1e3', status='READY', status_message=None, tags={}, user_id='', version='1'>

In [9]:
from mlflow.tracking import MlflowClient
client = MlflowClient()

client.transition_model_version_stage(
    name="EMI_Classifier_XGBoost",
    version=1,
    stage="Production"
)

client.transition_model_version_stage(
    name="EMI_Regressor_XGBoost",
    version=1,
    stage="Production"
)

C:\Users\Tasneem Inayath\AppData\Local\Temp\ipykernel_7196\515886856.py:4: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(
C:\Users\Tasneem Inayath\AppData\Local\Temp\ipykernel_7196\515886856.py:10: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1761767909435, current_stage='Production', deployment_job_state=<ModelVersionDeploymentJobState: current_task_name='', job_id='', job_state='DEPLOYMENT_JOB_CONNECTION_STATE_UNSPECIFIED', run_id='', run_state='DEPLOYMENT_JOB_RUN_STATE_UNSPECIFIED'>, description='', last_updated_timestamp=1761767954596, metrics=None, model_id=None, name='EMI_Regressor_XGBoost', params=None, run_id='c6efeb5a1e844a378d0b6051d1f52497', run_link='', source='models:/m-064cf115b7264f9ba051e9161f74b1e3', status='READY', status_message=None, tags={}, user_id='', version='1'>

In [11]:
import mlflow.pyfunc

classifier = mlflow.pyfunc.load_model("models:/EMI_Classifier_XGBoost/Production")
regressor = mlflow.pyfunc.load_model("models:/EMI_Regressor_XGBoost/Production")

# Predict
print(classifier.predict(pd.DataFrame(X_test[:1])))
print(regressor.predict(pd.DataFrame(X_test_reg[:1])))

[1]
[33660.38]


In [14]:
# Classification
print(classifier.predict(pd.DataFrame(X_test[10:11])))

# Regression
print(regressor.predict(pd.DataFrame(X_test_reg[10:11])))

[0]
[40796.004]


In [16]:
print(X_train.columns)

Index(['age', 'monthly_salary', 'years_of_employment', 'monthly_rent',
       'family_size', 'dependents', 'school_fees', 'college_fees',
       'travel_expenses', 'groceries_utilities', 'other_monthly_expenses',
       'existing_loans', 'current_emi_amount', 'credit_score', 'bank_balance',
       'emergency_fund', 'requested_amount', 'requested_tenure',
       'max_monthly_emi', 'total_expenses', 'debt_to_income_ratio',
       'expense_to_income_ratio', 'emi_gap', 'credit_risk_score',
       'employment_stability', 'education_High School',
       'education_Post Graduate', 'education_Professional',
       'employment_type_private', 'employment_type_self-employed',
       'emi_scenario_Education Emi', 'emi_scenario_Home Appliances Emi',
       'emi_scenario_Personal Loan Emi', 'emi_scenario_Vehicle Emi',
       'company_type_MNC', 'company_type_Mid-size', 'company_type_Small',
       'company_type_Startup', 'house_type_Own', 'house_type_Rented',
       'marital_status_Single', 'gender_

In [19]:
custom_applicant = custom_applicant.astype({
    'age': 'float',
    'monthly_salary': 'float',
    'years_of_employment': 'float',
    'monthly_rent': 'float',
    'school_fees': 'float',
    'college_fees': 'float',
    'travel_expenses': 'float',
    'groceries_utilities': 'float',
    'other_monthly_expenses': 'float',
    'current_emi_amount': 'float',
    'credit_score': 'float',
    'bank_balance': 'float',
    'emergency_fund': 'float',
    'requested_amount': 'float',
    'max_monthly_emi': 'float',
    'total_expenses': 'float',
    'debt_to_income_ratio': 'float',
    'expense_to_income_ratio': 'float',
    'emi_gap': 'float',
    'credit_risk_score': 'float',
    'employment_stability': 'float',
    'salary_credit_interaction': 'float',
    'balance_emi_gap': 'float',
    'family_size': 'int',
    'dependents': 'int',
    'existing_loans': 'int',
    'requested_tenure': 'int'
})

In [20]:
bool_columns = [col for col in custom_applicant.columns if 'education_' in col or
                'employment_type_' in col or 'emi_scenario_' in col or
                'company_type_' in col or 'house_type_' in col or
                'marital_status_' in col or 'gender_' in col]

custom_applicant[bool_columns] = custom_applicant[bool_columns].astype(bool)

In [21]:
eligibility = classifier.predict(custom_applicant)[0]
max_emi = regressor.predict(custom_applicant)[0]

print(f"Predicted Eligibility Class: {eligibility}")
print(f"Predicted Max EMI: ₹{max_emi:,.2f}")

2025/10/30 01:40:05 WARNING mlflow.models.utils: Found extra inputs in the model input that are not defined in the model signature: `['max_monthly_emi']`. These inputs will be ignored.


Predicted Eligibility Class: 2
Predicted Max EMI: ₹4,803.41


In [3]:
import pandas as pd

# Load your dataset
df = pd.read_csv("emi_prediction_dataset.csv")  # Replace with your actual file path

# Print all column names
print("All Columns:")
print(df.columns.tolist())

# Print unique values for categorical columns (up to 10 values each)
print("\nUnique values for categorical-like columns:")
for col in df.columns:
    if df[col].dtype == 'object' or df[col].nunique() < 20:
        print(f"{col}: {df[col].dropna().unique().tolist()[:10]}")

C:\Users\Tasneem Inayath\AppData\Local\Temp\ipykernel_5480\3490867906.py:4: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("emi_prediction_dataset.csv")  # Replace with your actual file path


All Columns:
['age', 'gender', 'marital_status', 'education', 'monthly_salary', 'employment_type', 'years_of_employment', 'company_type', 'house_type', 'monthly_rent', 'family_size', 'dependents', 'school_fees', 'college_fees', 'travel_expenses', 'groceries_utilities', 'other_monthly_expenses', 'existing_loans', 'current_emi_amount', 'credit_score', 'bank_balance', 'emergency_fund', 'emi_scenario', 'requested_amount', 'requested_tenure', 'emi_eligibility', 'max_monthly_emi']

Unique values for categorical-like columns:
age: [38.0, 58.0, 48.0, 32.0, 27.0, 47.0, 37.0, 31.0, 59.0, 49.0]
gender: ['Female', 'Male', 'female', 'male', 'M', 'MALE', 'F', 'FEMALE']
marital_status: ['Married', 'Single']
education: ['Professional', 'Graduate', 'High School', 'Post Graduate']
monthly_salary: ['82600.0', '21500.0', '86100.0', '66800.0', '57300.0', '38800.0', '27100.0', '392044.0', '47700.0', '129200.0']
employment_type: ['Private', 'Government', 'Self-employed']
company_type: ['Mid-size', 'MNC', 'St

In [ ]:
import pandas as pd

# Load your cleaned EMI dataset
df = pd.read_csv("datasets/encoded_data.csv")  # Replace with your actual path

# Step 1: Print all column names
print("📌 All Columns:")
print(df.columns.tolist())

# Step 2: Print data types
print("\n📊 Column Data Types:")
print(df.dtypes)

# Step 3: Print unique values for categorical-like columns
print("\n🔍 Unique Values for Categorical Columns:")
for col in df.columns:
    if df[col].dtype == 'object' or df[col].nunique() <= 20:
        print(f"{col}: {df[col].dropna().unique().tolist()}")

📌 All Columns:
['age', 'monthly_salary', 'years_of_employment', 'monthly_rent', 'family_size', 'dependents', 'school_fees', 'college_fees', 'travel_expenses', 'groceries_utilities', 'other_monthly_expenses', 'existing_loans', 'current_emi_amount', 'credit_score', 'bank_balance', 'emergency_fund', 'requested_amount', 'requested_tenure', 'emi_eligibility', 'max_monthly_emi', 'total_expenses', 'debt_to_income_ratio', 'expense_to_income_ratio', 'emi_gap', 'credit_risk_score', 'employment_stability', 'education_High School', 'education_Post Graduate', 'education_Professional', 'employment_type_private', 'employment_type_self-employed', 'emi_scenario_Education Emi', 'emi_scenario_Home Appliances Emi', 'emi_scenario_Personal Loan Emi', 'emi_scenario_Vehicle Emi', 'company_type_MNC', 'company_type_Mid-size', 'company_type_Small', 'company_type_Startup', 'house_type_Own', 'house_type_Rented', 'marital_status_Single', 'gender_FEMALE', 'gender_Female', 'gender_M', 'gender_MALE', 'gender_Male', 'g

In [9]:
import mlflow
import mlflow.xgboost
from xgboost import XGBClassifier

mlflow.set_experiment("emi_prediction_experiments")

with mlflow.start_run(run_name="XGB_Classifier_with_SHAP"):
    xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')
    xgb_model.fit(X_train_bal, y_train_bal)
    y_pred = xgb_model.predict(X_test)
    y_pred_proba = xgb_model.predict_proba(X_test)
    # log hyperparameters
    mlflow.log_param("max_depth", xgb_model.get_params()["max_depth"])
    mlflow.log_param("n_estimators", xgb_model.get_params()["n_estimators"])
    
    # log and register in one step with XGBoost flavor
    mlflow.xgboost.log_model(
        xgb_model,
        artifact_path="model",
        registered_model_name="EMI_Classifier_XGBoost"
    )

c:\Projects\emi_predict_ai\.venv\Lib\site-packages\xgboost\training.py:199: UserWarning: [23:20:36] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
2025/10/30 23:20:47 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
c:\Projects\emi_predict_ai\.venv\Lib\site-packages\xgboost\sklearn.py:1115: UserWarning: [23:20:47] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\c_api\c_api.cc:1575: Saving model in the UBJSON format as default.  You can use a file extension: `json` or `ubj` to choose between formats.
  self.get_booster().save_model(fname)
2025/10/30 23:20:55 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'EMI_Classifier_XGBoost' already exists. Creating a new version of this model...
C

In [11]:
from mlflow.tracking import MlflowClient

client = MlflowClient()

# Example: delete version 3 of your classifier
client.delete_model_version(
    name="EMI_Classifier_XGBoost",
    version=2
)

In [12]:
from mlflow.tracking import MlflowClient
client = MlflowClient()
versions = client.search_model_versions("name='EMI_Classifier_XGBoost'")
for v in versions:
    print(v.version, v.current_stage)

1 Archived


In [13]:
from mlflow.tracking import MlflowClient

client = MlflowClient()
client.transition_model_version_stage(
    name="EMI_Classifier_XGBoost",
    version=1,          # your archived version
    stage="Production"
)

C:\Users\Tasneem Inayath\AppData\Local\Temp\ipykernel_24464\1161345862.py:4: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1761767906236, current_stage='Production', deployment_job_state=None, description='', last_updated_timestamp=1761851109111, metrics=[<Metric: dataset_digest=None, dataset_name=None, key='Accuracy', model_id='m-489da889889443a8af1433bbbad454b8', run_id='cfe97507737c468d96328c44d88ba10e', step=0, timestamp=1761765894106, value=0.9825203460101346>,
 <Metric: dataset_digest=None, dataset_name=None, key='F1_Score', model_id='m-489da889889443a8af1433bbbad454b8', run_id='cfe97507737c468d96328c44d88ba10e', step=0, timestamp=1761765894139, value=0.9833273662190362>,
 <Metric: dataset_digest=None, dataset_name=None, key='Precision', model_id='m-489da889889443a8af1433bbbad454b8', run_id='cfe97507737c468d96328c44d88ba10e', step=0, timestamp=1761765894119, value=0.984853183694632>,
 <Metric: dataset_digest=None, dataset_name=None, key='Recall', model_id='m-489da889889443a8af1433bbbad454b8', run_id='cfe97507737c468d96328c44d88ba10e', step=0, timestamp=17